<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Setup-Super-Resolution" data-toc-modified-id="Setup-Super-Resolution-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setup Super-Resolution</a></span></li><li><span><a href="#Cyclical-Feeding" data-toc-modified-id="Cyclical-Feeding-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cyclical Feeding</a></span><ul class="toc-item"><li><span><a href="#TODOs" data-toc-modified-id="TODOs-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>TODOs</a></span></li></ul></li><li><span><a href="#Image-Sharpening" data-toc-modified-id="Image-Sharpening-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Image Sharpening</a></span></li><li><span><a href="#Source-Data-FaceSwap-and-Upscaling" data-toc-modified-id="Source-Data-FaceSwap-and-Upscaling-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Source Data FaceSwap and Upscaling</a></span></li><li><span><a href="#Celeba-Test" data-toc-modified-id="Celeba-Test-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Celeba Test</a></span></li><li><span><a href="#B/W-to-Color" data-toc-modified-id="B/W-to-Color-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>B/W to Color</a></span></li></ul></div>

# Intro
Notebook exploring random experiments around the use of the trained Faceswap generators.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt

from pathlib import Path
import sys
import pickle
import yaml
from numpy.random import shuffle
from ast import literal_eval
import tensorflow as tf

import cv2

from tqdm import tqdm

# Plotting
%matplotlib notebook
#%matplotlib inline

sns.set_context("paper")
sns.set_style("dark")

sys.path.append('../face_swap')

from utils import image_processing
from utils import super_resolution

from face_swap.deep_swap import swap_faces, Swapper
from face_swap import faceswap_utils as utils
from face_swap.plot_utils import stack_images
from face_swap import FaceGenerator, FaceDetector
from face_swap.train import get_original_data
from face_swap import gan, gan_utils
from face_swap import CONFIG_PATH
from face_swap.Face import Face

%load_ext autoreload
%autoreload 2

In [ ]:
data_folder = Path.home() / "Documents/datasets/"
models_folder = Path.home() / "Documents/models/"

# Load Data

In [ ]:
# Load two random celeba faces
from_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "img_align_celeba" / 
                            "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                             cv2.COLOR_BGR2RGB)
to_face_img = cv2.cvtColor(cv2.imread(str(data_folder / "img_align_celeba" / 
                          "000{}{}{}.jpg".format(*np.random.randint(0, 9, 3)))),
                       cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(from_face_img)
plt.show()
plt.imshow(to_face_img)
plt.show()

# Cyclical Feeding
Cycling feeding own output to generator. Can start with actual face or random noise. 

## TODOs
* Try apply text on image before feeding to generator

In [ ]:
def crop(img, crop_factor=0.2):
    h, w = img.shape[:2]
    h_crop = int((h * crop_factor)//2)
    w_crop = int((w * crop_factor)//2)
    return img[h_crop:h-h_crop, w_crop:w-w_crop]

In [ ]:
def zoom(img, zoom_factor=1.5):
    h, w = img.shape[:2]
    mat = cv2.getRotationMatrix2D((w//2, h//2), 0, zoom_factor)
    #mat[:, 2] -= (w//2, h//2)
    result = cv2.warpAffine(img, mat, (w, h), borderMode=cv2.BORDER_REPLICATE)
    return result

In [ ]:
# load config
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
model_cfg = cfg['masked_gan']['v1']

In [ ]:
# load generator and related functions
gen_a, gen_b, _, _ = gan.get_gan(model_cfg, load_discriminators=False)
_, _, _, fun_generate_a, fun_mask_a, fun_abgr_a = gan_utils.cycle_variables_masked(gen_a)
_, _, _, fun_generate_b, fun_mask_b, fun_abgr_b = gan_utils.cycle_variables_masked(gen_b)

In [ ]:
gen_fun_a = lambda x: fun_abgr_a([np.expand_dims(x, 0)])[0][0]
gen_fun_b = lambda x: fun_abgr_b([np.expand_dims(x, 0)])[0][0]

In [ ]:
generator_a = FaceGenerator.FaceGenerator(
            lambda face_img: FaceGenerator.gan_masked_generate_face(gen_fun_a, face_img),
            input_size=(64, 64), tanh_fix=True)
generator_b = FaceGenerator.FaceGenerator(
            lambda face_img: FaceGenerator.gan_masked_generate_face(gen_fun_b, face_img),
            input_size=(64, 64), tanh_fix=True)

In [ ]:
gen_input = Face(img, img)
use_a = True
generator = generator_a if use_a else generator_b
for i in range(500):
    out = get_hr_version(sr_model, generator.generate(gen_input, (64, 64))[0])
    #out = generator.generate(gen_input, (128, 128))[0]
    gen_input.face_img = FaceGenerator.random_transform(out, **cfg['random_transform'])
    #gen_input.img = zoom(out)
    res_path = str(data_folder / 'faceswap_experiments/cycle_feed/02/_{:04d}.png'.format(i))
    #cv2.imwrite(res_path, zoom(out))
    cv2.imwrite(res_path, out)
    # swap generator randomly every epoch
    #generator = generator_a if np.random.rand() > 0.5 else generator_b
    # swap generator every N epoch
    if i%50 == 0:
        use_a = not use_a
        generator = generator_a if use_a else generator_b

# Image Sharpening

In [ ]:
# adapted from https://github.com/AdityaPokharel/Sharpen-Image
regular_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
edge_enhance_kernel = np.array([[-1,-1,-1,-1,-1],
                               [-1,2,2,2,-1],
                               [-1,2,8,2,-1],
                               [-2,2,2,2,-1],
                               [-1,-1,-1,-1,-1]])/8.0
def sharpen(img, kernel=regular_kernel):
    # apply kernel to input image
    res = cv2.filter2D(img, -1, kernel)
    return res

# see also cv2.detailEnhance(src, sigma_s=10, sigma_r=0.15)

In [ ]:
plt.imshow(sharpen(to_face_img))
plt.show()

# Source Data FaceSwap and Upscaling
Try to cherry pick some results of face-swapping on the training data, apply upscaling to a reasonable size (e.g. 128x128) and any possible post-processing that might help in improving image quality.


In [ ]:
input_path = data_folder / "facesets" / "cage"
out_path = data_folder / "faceswap_experiments" / "source_faceswap" / "cage_trump"

out_size = (64, 64)

In [ ]:
# collected all image paths
img_paths = image_processing.get_imgs_paths(input_path, as_str=False)

# iterate over all collected image paths
for i, img_path in enumerate(img_paths):
    img = cv2.imread(str(img_path))
    gen_input = Face(img, img)
    gen_face = generator_b.generate(gen_input)[0]
    gen_face = sharpen(gen_face)
    gen_face = cv2.resize(gen_face, out_size)
    cv2.imwrite(str(out_path / "out_{:04d}.jpg".format(i)),
                            gen_face)

# Celeba Test
Test Celeba training and generation of artworks

In [ ]:
def plot_sample(images: list, predict_fun,
                tanh_fix=False, save_to: str=None, 
                nb_test_imgs=14, nb_columns=3, white_border=3):
    # need number of images divisible by number of columns
    nb_rows = nb_test_imgs//nb_columns
    assert nb_test_imgs % nb_columns == 0
    images = images[0:nb_test_imgs]

    figure = np.stack([
                        images,
                        predict_fun(images),
                        ], axis=1)
    # we split images on two columns
    figure = figure.reshape((nb_columns, nb_rows) + figure.shape[1:])
    figure = stack_images(figure)
    img_width = images[0].shape[1]
    img_height = images[0].shape[0]
    for i in range(1, nb_columns):
        x = img_width*2*i
        figure[:, x-white_border:x+white_border, :] = 255.0
    for i in range(1, nb_rows):
        y = img_height*i
        figure[y-white_border:y+white_border, :, :] = 255.0

    if save_to:
        cv2.imwrite(save_to, figure)
    else:
        figure = cv2.cvtColor(figure, cv2.COLOR_BGR2RGB)
        #plt.imshow(figure)
        #plt.show()
        display(Image.fromarray(figure))
        # crashes in notebooks
        #cv2.imshow('', figure)
        #cv2.waitKey(0)

In [ ]:
# load config
with open(CONFIG_PATH, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)
model_cfg = cfg['masked_gan']['v1']
model_cfg['models_path'] = str(models_folder / "face_recognition/deep_faceswap/masked_gan/cage_celeba/v4")

In [ ]:
#tf.reset_default_graph()
face_detector = FaceDetector.FaceDetector(cfg)

In [ ]:
# load generator and related functions
netGA, netGB, _, _ = gan.get_gan(model_cfg, load_discriminators=False)

# define generation and plotting function
# depending if using masked gan model or not
if model_cfg['masked']:
  distorted_A, fake_A, mask_A, path_A, fun_mask_A, fun_abgr_A = gan_utils.cycle_variables_masked(netGA)
  distorted_B, fake_B, mask_B, path_B, fun_mask_B, fun_abgr_B = gan_utils.cycle_variables_masked(netGB)
  #gen_plot_a = lambda x: np.array(path_A([x])[0]) 
  #gen_plot_b = lambda x: np.array(path_B([x])[0])
  gen_plot_a = lambda x: np.array(fun_abgr_A([x])[0][ :, :, :, 1:]) 
  gen_plot_b = lambda x: np.array(fun_abgr_B([x])[0][ :, :, :, 1:])
  gen_plot_mask_a = lambda x: np.array(fun_mask_A([x])[0])*2-1
  gen_plot_mask_b = lambda x: np.array(fun_mask_B([x])[0])*2-1
else:
  gen_plot_a = lambda x: netGA.predict(x)
  gen_plot_b = lambda x: netGB.predict(x)

In [ ]:
sr_model = super_resolution.get_SRResNet(cfg['super_resolution'])
resize_fun = lambda img, size: FaceGenerator.super_resolution_resizing(sr_model, img, size)

In [ ]:
gen_fun_a = lambda x: fun_abgr_A([np.expand_dims(x, 0)])[0][0]
gen_fun_b = lambda x: fun_abgr_B([np.expand_dims(x, 0)])[0][0]
gen_input_size = literal_eval(model_cfg['img_shape'])[:2]
face_generator = FaceGenerator.FaceGenerator(
    lambda face_img: FaceGenerator.gan_masked_generate_face(gen_fun_a, face_img),
    input_size=gen_input_size, config=cfg['swap'], resize_fun=resize_fun)

In [ ]:
swapper = Swapper(face_detector, face_generator, cfg['swap'], save_all=True)

In [ ]:
def swap(img):
    face = Face(img.copy(), Face.Rectangle(0, 64, 64, 0))
    #return swap_faces(face, face_detector, cfg['swap'], face_generator)
    return face.get_face_img()
#gen_plot_b = lambda x: [swap(img) for img in x]
gen_plot = lambda x: [swapper.swap(img) for img in x]

In [ ]:
img_dir_a = data_folder / 'facesets/cage'
img_dir_b = data_folder / 'celeba_tmp'
#images_a, images_b = get_original_data(img_dir_a, img_dir_b, img_size=None, tanh_fix=False)
images = image_processing.load_data(image_processing.get_imgs_paths(img_dir_a), (128, 128))

In [ ]:
dest_folder = str(data_folder / "faceswap_experiments/source_faceswap/cage_celeba_masked/test_1/_{}.png")
swapper.config['mask_method'] = "gen_mask"
face_generator.border_expand = (0.1, 0.1)
face_generator.blur_size = 13
face_generator.align = False
#shuffle(images)
for i in range(20):
    print(i)
    images_subset = images[i*15:(i+1)*15]
    try:
        plot_sample(images_subset, gen_plot, nb_test_imgs=15, nb_columns=3, 
                    save_to=dest_folder.format(i), tanh_fix=False)
    except FaceDetector.FaceSwapException:
        pass